In [1]:
import pandas as pd

dim_time = pd.read_csv(r"powerbi_parquet\dim_time.csv", dtype={"date_key": "Int64"})
fact_fire = pd.read_csv(r"powerbi_parquet\fact_incident_fire.csv", dtype={"incident_date_key": "Int64"})

In [2]:
(
    dim_time["date_key"].isna().sum(),
    dim_time["date_key"].duplicated().sum(),
    fact_fire["incident_date_key"].isna().sum(),
)

(np.int64(0), np.int64(0), np.int64(0))

In [3]:
dim_keys = set(dim_time["date_key"].dropna())
unmatched = fact_fire.loc[
    fact_fire["incident_date_key"].notna()
    & ~fact_fire["incident_date_key"].isin(dim_keys),
    "incident_date_key"
]

len(unmatched), unmatched.unique()[:10]


(0,
 <IntegerArray>
 []
 Length: 0, dtype: Int64)

In [4]:
(
    dim_time["year"].isna().sum(),
    ((dim_time["date_key"] // 10000) != dim_time["year"]).sum(),
    sorted((fact_fire["incident_date_key"] // 10000).dropna().unique())[:10]
)


(np.int64(0),
 np.int64(0),
 [np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)])

In [ ]:
YEAR = 2022

dim = dim_time.copy()
fact = fact_fire.copy()

df = fact.merge(dim[["date_key","year","month","day"]], left_on="incident_date_key", right_on="date_key", how="left")
df = df[df["year"] == YEAR]

hm = df.pivot_table(index="month", columns="day", values="incident_date_key", aggfunc="count", fill_value=0).sort_index()
hm
